created by j.han: tfnotebooks@gmail.com

# params and directories

this notebook assumes the following directory structure:

/content

    |--- /gdrive
            |--- /My Drive
                    |--- /TensorFlow
                            |---/frozen_models
                                    |---/ssd
                                          |---your frozen model.pb

In [0]:
pb_path='_______________________________'             #name of your frozen model.pb upload it to according to the directory shown above
link='_________________________'                      #the forward url from ngrok that captures the video stream 
threshold=0.5                                         #prediction thereshold
authtoken = '_______________________' 
#authtoken for the ngrok account used to forward this machine. note that this is a separate account from the one used to forward the raspberry pi videostream

# checking GPU specs
T4 would be the best, but unlikely to get since Colab frowns upon this kind of usage for its gpu. Most likely will get K80, still decent enough 


In [9]:
!nvidia-smi
!/usr/local/cuda/bin/nvcc --version

Sat Sep 14 00:13:45 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# ngrok tunneling setup

credit to [Imad El-Hanafi](https://imadelhanafi.com/posts/google_colal_server/)
for his great work on SSH into the Colab virtual machines



In [10]:
#mounts the drive
from google.colab import drive                                                          
drive.mount('/content/gdrive/')

#sets up ssh
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(5))
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
get_ipython().system_raw('/usr/sbin/sshd -D &')
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

# opens a tcp tunnel with ngrok so that the terminal of the virtual machine that this colab session is running on can be remotely accessed
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
print('root')
print(password)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
root
g9cch
Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
root
kqmIU


now go to this [link](https://dashboard.ngrok.com/status) to open up the ngrok dashboard. note the port number.


# Change directories
navigate to the same directory as uploaded frozen model.pb


In [11]:
%cd ~
%cd /content/gdrive/'My Drive'/TensorFlow/frozen_models/ssd

/root
/content/gdrive/My Drive/TensorFlow/frozen_models/ssd
/root
/content/gdrive/My Drive/TensorFlow/frozen_models/ssd


# frameGrabber class

since the video streamed is acutally MJPEGs, the default opencv videocapture class wouldn't work. Instead, use GET method from the requests module to pull the image bytes directly. Multi-threaded for reduced I/O latency. Inspired by Zaw Lin's answer in this [stackoverflow post](https://stackoverflow.com/questions/21702477/how-to-parse-mjpeg-http-stream-from-ip-camera/21844162)

In [0]:
from threading import Thread  
import cv2
import requests
import numpy as np

class frameGrabber:
    def __init__(self,url):
        self.r=requests.get(url,stream=True)                                    #GET method to the video streaming url
        self.img=None                                                           #current image
        self.stopped=False                                                      #whether to stop grabbing
        print(self.r.status_code)                                               #200 is good  
                  
        
    def start(self):
        print('starting thread')
        t=Thread(target=self.update,args=())                                  
        t.setDaemon(True)
        t.start()
        print('thread started')
        return self
       
    def update(self):
        while True:
            
            if self.stopped==True:
                return
            else:
                img_bytes=bytes()
                for chunk in self.r.iter_content(chunk_size=1024):
                    img_bytes+=chunk
                    a=img_bytes.find(b'\xff\xd8')                               #jpg on page always starts with these bytes
                    b=img_bytes.find(b'\xff\xd9')                               #jpg on page always ends with these bytes
                    if a!=-1 and b!=-1:
                        jpg=img_bytes[a:b+2]
                        img_bytes=img_bytes[b+2:]
                        self.img=cv2.imdecode(np.frombuffer(jpg,dtype=np.uint8),cv2.IMREAD_COLOR) #byts to np array
    def read(self):
        return self.img
    
    def stop(self):
        self.stopped=True

# model

standard setup of tf model

model outputs '<x1,y1,x2,y2,class label,confidence,fps>' upon detection 

In [13]:
from flask import Flask, Response
import tensorflow as tf
import cv2 
import numpy as np
import time
import argparse
import sys

def predict(img,sess,label_map):
    rows = img.shape[0]
    cols = img.shape[1]
    img_expanded=np.expand_dims(img,axis=0)
    
    start=time.time()
    out = sess.run([sess.graph.get_tensor_by_name('num_detections:0'),
                    sess.graph.get_tensor_by_name('detection_scores:0'),
                    sess.graph.get_tensor_by_name('detection_boxes:0'),
                    sess.graph.get_tensor_by_name('detection_classes:0')],
                   feed_dict={'image_tensor:0': img_expanded})
    elapsed=time.time()-start
    if elapsed>0:
          fps=int(1/elapsed)
        
    top_five=out[1][0][0:5]
    top_five=[round(100*i,2) for i in top_five]
     
    classId = int(out[3][0][0])
    score = float(out[1][0][0])
    bbox = [float(v) for v in out[2][0][0]]
    if score > threshold:
        x = bbox[1] * cols
        y = bbox[0] * rows
        right = bbox[3] * cols
        bottom = bbox[2] * rows
        label=label_map[classId]
        info='<'+str(round(x))+','+str(round(y))+','+str(round(right))+','+str(round(bottom))+','+str(label)+','+str(round(score,2))+','+str(fps)+'>'
        return info
    else:
      return '<pass>'


label_map= {0: 'background',
              1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane', 6: 'bus',
              7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light', 11: 'fire hydrant',
              13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'cat',
              18: 'dog', 19: 'horse', 20: 'sheep', 21: 'cow', 22: 'elephant', 23: 'bear',
              24: 'zebra', 25: 'giraffe', 27: 'backpack', 28: 'umbrella', 31: 'handbag',
              32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard',
              37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove',
              41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle',
              46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon',
              51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange',
              56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut',
              61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed',
              67: 'dining table', 70: 'toilet', 72: 'tv', 73: 'laptop', 74: 'mouse',
              75: 'remote', 76: 'keyboard', 77: 'cell phone', 78: 'microwave', 79: 'oven',
              80: 'toaster', 81: 'sink', 82: 'refrigerator', 84: 'book', 85: 'clock',
              86: 'vase', 87: 'scissors', 88: 'teddy bear', 89: 'hair drier', 90: 'toothbrush'}

cap=frameGrabber(link).start()

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(pb_path, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


200
starting thread
thread started
404
starting thread
thread started


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-12-417be9e40739>", line 29, in update
    for chunk in self.r.iter_content(chunk_size=1024):
  File "/usr/local/lib/python3.6/dist-packages/requests/models.py", line 769, in iter_content
    raise StreamConsumedError()
requests.exceptions.StreamConsumedError



# web server

starts web service to upload each frame's prediction information at port 5000 of the local network.
the uploaded info is either  '<x1,y1,x2,y2,class label,confidence,fps>' or pass otherwise

This info is NOT accessible until start third ngrok session. See cell below on instruction


In [7]:
app = Flask(__name__)

@app.route('/')
def index():
    return Response(gen(detection_graph,label_map),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

def gen(detection_graph,label_map):
    print('start prediction')
    print()

    with detection_graph.as_default():
        with tf.compat.v1.Session(graph=detection_graph) as sess:
            while True:
#                 start=time.time()
                img=cap.read()
                info=predict(img,sess,label_map)
                if info is not None:
                  yield (info)
                else:
                  yield ('<pass>')

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


start prediction



127.0.0.1 - - [14/Sep/2019 00:15:25] "GET / HTTP/1.1" 200 -
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-417be9e40739>", line 29, in update
    for chunk in self.r.iter_content(chunk_size=1024):
  File "/usr/local/lib/python3.6/dist-packages/requests/models.py", line 769, in iter_content
    raise StreamConsumedError()
requests.exceptions.StreamConsumedError



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [0]:
# go to https://dashboard.ngrok.com/status
# note the port number (ex for 0.tcp.ngrok.io: 17129, port number would be 17129)
# (linux, iOS): use ssh at terminal to connect to the remote machine's command line via the username and password generated in the ngrok tunneling stup section
# (winodws): use PuTTY with the same username and password

# type the following:
# cd /content
# ./ngrok authtoken (paste the authtoken from your 3rd ngrok account)
# ./ngrok http 5000

# you will now see something like below
# keep this window open and note the http://*****.ngrok.io/  address that the colab server is forwarded to
# now go to the local machine and run client.py with the address 
